In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%load_ext autoreload
%autoreload 2

In [114]:
from os                import walk
from pydub             import AudioSegment
from pydub.utils       import get_array_type
from pydub.utils       import mediainfo
from pydub.silence     import split_on_silence
from datasets          import load_dataset
from torchaudio.utils  import download_asset
from scipy             import signal
from scipy.io          import wavfile
from matplotlib.pyplot import figure
from tqdm              import tqdm
from os                import listdir
from os.path           import isfile, join
from datetime          import datetime
from pyctcdecode       import build_ctcdecoder
from pprint            import pprint
from torch.utils.data  import Dataset, DataLoader

import matplotlib.pyplot as plt
import pandas            as pd
import numpy             as np
import soundfile         as sf

import re
import json
import requests
import whisper
import git
import os
import jiwer
import IPython
import array
import librosa
import torch
import torchaudio

<h1 style="background-color:#28AA9C;"> <center> <a id='start_cell'></a> Table of content: </center></h1>

[Pipeline](#pipeline_cell) </br>
[Police Example](#Police_Example) </br>
[VAD Threshold paramter example](#Threshold_Example) </br>
[Whisper](#Whisper) </br>
[Police Whisper](#police_dataset_whisper_cell) </br>

<h1 style="background-color:#EBA11F;"> <center> Hebrew: word reading rate:  </center></h1>

https://www.globes.co.il/news/article.aspx?did=603261

<p style="color:red;"> 140-150 Words per minute -> seem unreal -> so I took 1.5 words per second </p>


<h1 style="background-color:LightGreen;"> <center> GPU / CPU </center></h1>

In [31]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

<h1 style="background-color:LightGreen;"> <center> <a id='pipeline_cell'></a> Pipeline Utils </center></h1>

[Table Of Content](#start_cell)

In [5]:
def get_total_speaking_length(speech_timestamps):
    total = 0
    for val in speech_timestamps:
        tmp = val['end'] - val['start']
        total = total + tmp
    return total

In [115]:
def HebrewNormalizer(hebrew_text):
    # --- step 1: remove sign characters
    #
    # ignore_characters = ",~!@#%^&*()-+/|<>[]*'?.{}"
    # for character in ignore_characters:
    #     hebrew_text = hebrew_text.replace(character, '')

        # --- step 2: replace signs
    hebrew_text = hebrew_text.replace('$', " דולר")
    hebrew_text = hebrew_text.replace('₪', " שח")
    hebrew_text = hebrew_text.replace('€', " יורו")
    # hebrew_text = hebrew_text.replace('.', " נקודה")
    hebrew_text = hebrew_text.replace('ת"א', "תל אביב")
    hebrew_text = hebrew_text.replace('ב"ש', "באר שבע")
    hebrew_text = hebrew_text.replace('ע"י', "על ידי")
    hebrew_text = hebrew_text.replace('אח"כ', "אחר כך")
    hebrew_text = hebrew_text.replace('\"', "")

    # for now we will not handle digits, we will have to handle digits if it costs us in the performance of the model
    # TODO: handle dates: 3/7 -> third of july
    # # --- step 3: replace numbers to words
    # dict_nums = {
    #     "0": "אפס",
    #     "1": "אחד",
    #     "2": "שתיים",
    #     "3": "שלוש",
    #     "4": "ארבע",
    #     "5": "חמש",
    #     "6": "שש",
    #     "7": "שבע",
    #     "8": "שמונה",
    #     "9": "תשע",
    #     "10": "עשר",
    # }
    # for digit, word in dict_nums.items():
    #     hebrew_text = hebrew_text.replace(digit, word)
    #
    # # --- step 4: replace female numbers to male numbers
    # dict_male = {
    #     "אחת": "אחד",
    #     "שתיים": "שניים",
    #     "שלושה": "שלוש",
    #     "ארבעה": "ארבע",
    #     "חמישה": "חמש",
    #     "שישה": "שש",
    #     "שבעה": "שבע",
    #     "תשעה": "תשע",
    #
    # }
    # for female, male in dict_male.items():
    #     hebrew_text = hebrew_text.replace(female, male)
    # postproccessing, removing special charcteres after handling and translating them
    valid_tokens = "פ ם ן ו ט א ר ק ף ך ל ח י ע כ ג ד ש ץ ת צ מ נ ה ב ס ז 1 2 3 4 5 6 7 8 9 0"
    valid_tokens = set([x.lower() for x in valid_tokens])
    # The caret in the character class ([^) means match anything but
    invalid_chars_regex = f"[^\s{re.escape(''.join(set(valid_tokens)))}]"


    """ DO ADAPT FOR YOUR USE CASE. this function normalizes the target text. """
    hebrew_text = re.sub(invalid_chars_regex, " ", hebrew_text)
    hebrew_text = re.sub("\s+", " ", hebrew_text).strip()
    # --- return result
    return hebrew_text    
   

In [244]:
def run_pipeline(vad_model, vad_utils, source_file, dest_file, text, sample_rate, mp3_format=False):
    
    # 1. get silero util functions
    (get_speech_timestamps, save_audio, read_audio,VADIterator, collect_chunks) = vad_utils
    
    # 2. get file VAD
    wav_audio         = read_audio(source_file, sampling_rate=sample_rate)       
    full_audio_lenth  = len(wav_audio) / sample_rate
    speech_timestamps = get_speech_timestamps(wav_audio, vad_model, sampling_rate=sample_rate, threshold=0.4) # see 'bad example' for threshold
    
    # 3. calc staicics
    new_file_length   = get_total_speaking_length(speech_timestamps) / sample_rate
    text_length       = len(text.split())
    if text_length < (1.2*new_file_length):
        file_name = source_file[source_file.rfind("/")+1:]
        print(f"Outlier: {file_name}, Text length: {text_length}, File Length: {full_audio_lenth} seconds, File Without Silence: {new_file_length}, Required Min Words: {(140/60*new_file_length):.2f}")
        return None, None
    
    # if we will used mp3
    if mp3_format:
        # 4. convert to new file (without noise)    
        save_audio("tmp.wav",collect_chunks(speech_timestamps, wav_audio), sampling_rate=sample_rate)
        
        # 5. convert to mp3
        AudioSegment.from_wav("tmp.wav").export(dest_file, format="mp3")
        os.remove("tmp.wav")
    else:
        save_audio(dest_file, collect_chunks(speech_timestamps, wav_audio), sampling_rate=sample_rate)

    
    return text_length, new_file_length
    

In [245]:
def run_on_new_police_data(vad_model, vad_utils, source_folder, dest_folder, sample_rate, gt_excell_path):
    
    # get poilce GT text
    df_gt                  = pd.read_excel(gt_excell_path)
    
    
    # get sub folder list
    folder_list   = [f for f in listdir(source_folder) if not isfile(join(source_folder, f))]        
    df_statistics = pd.DataFrame()

    files_name        = []    
    gt_len            = []
    file_length       = []      
    gt_text           = []
    
    for folder_name in tqdm(folder_list):
        
        # --- get wav file
        splitted_wav_folder = f"{source_folder}/{folder_name}"
        splitted_files      = [entry for entry in os.listdir(splitted_wav_folder) if os.path.isfile(os.path.join(splitted_wav_folder, entry))]
        
        # --- get GT text
        gt_list             = df_gt[df_gt['file'] == f"{folder_name}.mp3"]['text'].values                
                        
        for i in range(len(splitted_files)): 
            part           = "%04d" % (i,)
            full_src       = f"{source_folder}/{folder_name}/{folder_name}_{part}.wav"       
            full_dst       = f"{dest_folder}/{folder_name}_{part}.'wav'"       
            
            
            text_length, new_file_length = run_pipeline(vad_model, 
                                                        vad_utils, 
                                                        full_src, 
                                                        full_dst, 
                                                        HebrewNormalizer(gt_list[i]), 
                                                        sample_rate)
            # if outlier
            if text_length is None:
                continue
            
            gt_text.append(gt_list[i])
            files_name.append(f"{folder_name}_{part}")
            gt_len.append(text_length)
            file_length.append(new_file_length)        
                        

    df_statistics = pd.DataFrame(data = {"File":   files_name,
                                  "Duration":      file_length,
                                  "Text Length":   gt_len,
                                  "Text":          gt_text
                                  })                
    df_statistics.to_csv(f"{dest_folder}/statistics.csv", index=False)
    

<h1 style="background-color:LightGreen;"> <center> <a id='Police_Example'></a>  Example on new police corpus </center></h1>

[Table Of Content](#start_cell)

In [217]:
vad_model, vad_utils = torch.hub.load(repo_or_dir  = 'snakers4/silero-vad',
                                      model        = 'silero_vad',
                                      force_reload = True,
                                      onnx         = False)


Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /home/amitli/.cache/torch/hub/master.zip


In [13]:
gt_excell_path = "/home/amitli/Datasets/Unit_Tests/new_corpus.xlsx"
source_folder  = "/home/amitli/Datasets/Unit_Tests/Source"
dest_folder    = "/home/amitli/Datasets/Unit_Tests/Dest"
sample_rate    = 8000 

run_on_new_police_data(vad_model, vad_utils, source_folder, dest_folder, sample_rate, gt_excell_path)

In [193]:
df = pd.read_csv("/home/amitli/Datasets/Unit_Tests/Dest/statistics.csv")
df.head()

,File,Duration,Text Length,Text
0,-3dd58g7rQQ_0000,4.410,11,שלום ילדים וילדות אני איריס ארגמן אני סופרת יל...
1,-3dd58g7rQQ_0001,4.514,9,הספר הספר הזה קצת מיוחד ושונה מספרים אחרים שכתבתי
2,-3dd58g7rQQ_0002,3.890,9,במהלך העבודה שלי כסופרת כי הספר הזה מבוסס על
3,-3dd58g7rQQ_0003,4.476,12,פי סיפור אמיתי של איש אמיתי בשם פרד לסינג שחי ...
4,-3dd58g7rQQ_0004,4.362,9,מהולנד שחי שעבר להולנד וחי עד היום באמריקה את


<h1 style="background-color:#AA2849;"> <center> <a id='Threshold_Example'></a> Example why I used threshold=0.4 (instead of 0.5) </center></h1>

[Table Of Content](#start_cell)

<p style="color:green;">GT Text: "אולגה ובעלה יהושע אז בעצם מהי"</p>

In [34]:
IPython.display.Audio("/home/amitli/Datasets/Unit_Tests/Source/-37_ZAraHQM/-37_ZAraHQM_0011.wav")

<p style="color:red;">Result Text:  missing "יהושע"</p>

In [130]:
(get_speech_timestamps, save_audio, read_audio,VADIterator, collect_chunks) = vad_utils
sample_rate       = 8000
wav_audio         = read_audio("/home/amitli/Datasets/Unit_Tests/Source/-37_ZAraHQM/-37_ZAraHQM_0011.wav", sampling_rate=sample_rate)       
speech_timestamps = get_speech_timestamps(wav_audio, vad_model, sampling_rate=sample_rate)        
save_audio("/home/amitli/Datasets/Unit_Tests/tmp.wav",collect_chunks(speech_timestamps, wav_audio), sampling_rate=sample_rate)
IPython.display.Audio("/home/amitli/Datasets/Unit_Tests/tmp.wav")

<p style="color:blue;">Try to manual fix:</p>

In [131]:
(get_speech_timestamps, save_audio, read_audio,VADIterator, collect_chunks) = vad_utils
sample_rate       = 8000
wav_audio         = read_audio("/home/amitli/Datasets/Unit_Tests/Source/-37_ZAraHQM/-37_ZAraHQM_0011.wav", sampling_rate=sample_rate)       
speech_timestamps = get_speech_timestamps(wav_audio, vad_model, sampling_rate=sample_rate, threshold=0.4)        
save_audio("/home/amitli/Datasets/Unit_Tests/tmp.wav",collect_chunks(speech_timestamps, wav_audio), sampling_rate=sample_rate)
IPython.display.Audio("/home/amitli/Datasets/Unit_Tests/tmp.wav")

<h1 style="background-color:LightGreen;"> <center> <a id='Whisper'></a> Whisper </center></h1>

[Table Of Content](#start_cell)

In [247]:
def Get_Whisper_Text(whisper_model, file_name, device):
    
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(file_name)
    audio = whisper.pad_or_trim(audio)
    mel   = whisper.log_mel_spectrogram(audio).to(whisper_model.device)

    # decode the audio
    options = whisper.DecodingOptions(language = 'he', beam_size=8, patience=2, fp16 = False)    #, temperature=0.2
    result  = whisper.decode(whisper_model, mel, options)
    result  = result.text
    result  = HebrewNormalizer(result)
    
    return result

In [ ]:
whisper_model = whisper.load_model("large", device="cpu")

<p style="color:#28AA9C; font-size:24px;"> Run on first 2 rows from dataframe and add Whisper results: </p>

In [248]:
df             = pd.read_csv("/home/amitli/Datasets/Unit_Tests/Dest/statistics.csv")
whisper_column = [None]*df.shape[0]
whisper_wer    = [None]*df.shape[0]

for i in range(2):
    audio_file        = f"{dest_folder}/{df['File'].values[i]}.mp3"
    whisper_text      = Get_Whisper_Text(whisper_model, audio_file, "cpu")
    whisper_column[i] = (whisper_text)
    whisper_wer[i]    = jiwer.wer(df["Text"].values[i], whisper_text)
    
    
df["Whisper"] = whisper_column
df["WER"]     = whisper_wer

df.to_csv(f"{dest_folder}/whisper.csv", index=False)

In [249]:
df = pd.read_csv(f"{dest_folder}/whisper.csv")
df.head()

,File,Duration,Text Length,Text,Whisper,WER
0,-3dd58g7rQQ_0000,4.410,11,שלום ילדים וילדות אני איריס ארגמן אני סופרת יל...,שלום ילדים וילדות אני אירי שרגמן פרס ילדים ועו...,0.454545
1,-3dd58g7rQQ_0001,4.514,9,הספר הספר הזה קצת מיוחד ושונה מספרים אחרים שכתבתי,הספר הזה קצת מיוחד ושונה מספרים אחרי שכתבתי במ...,0.333333
2,-3dd58g7rQQ_0002,3.890,9,במהלך העבודה שלי כסופרת כי הספר הזה מבוסס על,NaN,NaN
3,-3dd58g7rQQ_0003,4.476,12,פי סיפור אמיתי של איש אמיתי בשם פרד לסינג שחי ...,NaN,NaN
4,-3dd58g7rQQ_0004,4.362,9,מהולנד שחי שעבר להולנד וחי עד היום באמריקה את,NaN,NaN


In [175]:
def Get_Whisper_Transcribe(whisper_model, file_name):            
    result = whisper_model.transcribe(file_name)
    return result['text']

<h1 style="background-color:#AA2849;"> <center> <a id='Threshold_Example'></a> How can it be ??? </center></h1>

In [209]:
police_source_file = "/home/amitli/Datasets/ASR_Police/new_corpus/-37_ZAraHQM.mp3"
print(f"Police File SR: {mediainfo(police_source_file)['sample_rate']}")

Police File SR: 8000


In [208]:
test_full_file = "/home/amitli/Datasets/Unit_Tests/Source/-3dd58g7rQQ/-3dd58g7rQQ_0001.wav"
mp3_file       = "/home/amitli/Datasets/test_mp3.mp3"
wav_file       = "/home/amitli/Datasets/test_wav.wav"

# 1. get source sample rate:
print(f"Source file SR: {mediainfo(test_full_file)['sample_rate']}")
    

# 2. save as mp3
run_pipeline(vad_model, vad_utils, test_full_file, mp3_file, "a b c d e f", sample_rate=8000, mp3_format=True)
print(f"MP3 file SR: {AudioSegment.from_mp3(mp3_file).frame_rate}")

# 3. save as wav
run_pipeline(vad_model, vad_utils, test_full_file, wav_file, "a b c d e f", sample_rate=8000, mp3_format=False)
print(f"WAV file SR: {mediainfo(wav_file)['sample_rate']}")

mp3_text = Get_Whisper_Text(whisper_model, mp3_file, "cpu")
wav_text = Get_Whisper_Text(whisper_model, wav_file, "cpu")

mp3_wer  = jiwer.wer(df["Text"].values[1], mp3_text)
wav_wer  = jiwer.wer(df["Text"].values[1], wav_text)

print(f"MP3 WER: {mp3_wer:.3f}")
print(f"WAV WER: {wav_wer:.3f}")

Source file SR: 16000
MP3 file SR: 8000
WAV file SR: 8000
MP3 WER: 0.333
WAV WER: 0.444


In [236]:
print(f"GT : {df['Text'].values[1]}")
print(f"MP3: {mp3_text}")
print(f"WAV: {wav_text}")

GT : הספר הספר הזה קצת מיוחד ושונה מספרים אחרים שכתבתי
MP3: הספר הזה קצת מיוחד ושונה מספרים אחרי שכתבתי במהלך.
WAV: הספר הזה קצת מיוחד, ושונה מספרים אחרי שכתבתי במהלך.


In [243]:
normalize_mp3 = HebrewNormalizer(mp3_text)
normalize_wav = HebrewNormalizer(wav_text)


print(f"GT : {df['Text'].values[1]}")
print(f"Normalized MP3: {normalize_mp3}")
print(f"Normalized WAV: {normalize_wav}")

mp3_wer  = jiwer.wer(df["Text"].values[1], normalize_mp3)
wav_wer  = jiwer.wer(df["Text"].values[1], normalize_wav)

print(f"MP3 WER: {mp3_wer:.3f}")
print(f"WAV WER: {wav_wer:.3f}")

GT : הספר הספר הזה קצת מיוחד ושונה מספרים אחרים שכתבתי
Normalized MP3: הספר הזה קצת מיוחד ושונה מספרים אחרי שכתבתי במהלך.
Normalized WAV: הספר הזה קצת מיוחד ושונה מספרים אחרי שכתבתי במהלך.
MP3 WER: 0.333
WAV WER: 0.333


<h1 style="background-color:#28AA9C;"> <center> <a id='police_dataset_whisper_cell'></a> Run Police Whisper </center></h1>

[Table Of Content](#start_cell)

In [23]:
df_gt         = pd.read_excel("/home/amitli/Datasets/MyVoiceTests_16SR/new_corpus.xlsx")
source_folder = "/home/amitli/Datasets/MyVoiceTests_16SR"

In [121]:
class PoliceDataset(Dataset):
        
    def __init__(self, source_folder, df_gt, device):        
        
        self.folder_list   = [f for f in listdir(source_folder) if not isfile(join(source_folder, f))]        
        self.l_main_file   = []
        self.l_sub_file    = []
        self.l_full_path   = []
        self.l_gt          = []
        self.device        = device

        for folder_name in tqdm(folder_list):

            # --- get wav file
            splitted_wav_folder = f"{source_folder}/{folder_name}"
            splitted_files      = [entry for entry in os.listdir(splitted_wav_folder) if os.path.isfile(os.path.join(splitted_wav_folder, entry))]

            # --- get GT text
            gt_list             = df_gt[df_gt['file'] == f"{folder_name}.mp3"]['text'].values                

            for i in range(len(splitted_files)): 
                part           = "%04d" % (i,)
                full_src       = f"{source_folder}/{folder_name}/{folder_name}_{part}.wav"       

                self.l_main_file.append(folder_name)
                self.l_sub_file.append(f"{folder_name}_{part}.wav")
                self.l_full_path.append(full_src)                
                self.l_gt.append(gt_list[i])
        

    def __len__(self):
        return len(self.l_gt)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        audio_file_path = l_full_path[idx]
        audio           = whisper.load_audio(str(audio_file_path))
        audio           = whisper.pad_or_trim(audio)
        
        mel             = whisper.log_mel_spectrogram(audio).to(self.device)        
        
        sample          = {'mel':      mel, 
                          'text':      HebrewNormalizer(self.l_gt[idx]), 
                          'main_file': self.l_main_file[idx],
                          'sub_file':  self.l_sub_file[idx],
                          'full_path': audio_file_path}
        
        return sample

In [122]:
dataset = PoliceDataset(source_folder = "/home/amitli/Datasets/MyVoiceTests_16SR",
                  df_gt         = df_gt,
                  device        = DEVICE)

loader = DataLoader(dataset, batch_size=20, shuffle=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 555.03it/s]


In [123]:
len(dataset)

23596

In [136]:
df = pd.DataFrame()
for batch in tqdm(loader):
    
    languages        = []
    mel              = batch['mel']
    audio_data       = {'wav': json.dumps(mel.tolist()), 'languages': languages}
    gt               = batch['text']
            
    res              = requests.get('http://10.53.140.33:80/batch_inference/', json=audio_data)
    res_list         = res.json()[0]
    
    l_wer            = []
    l_whisper        = []    
    l_res_lang       = []
    l_avg_logprob    = []
    l_no_speech_prob = []
    l_compres_ratio  = []
        
    for i, res in enumerate(res_list):
        whisper_text = res['text']
        whisper_text = HebrewNormalizer(whisper_text)
        l_whisper.append(whisper_text)                
        if whisper_text == '':
            l_wer.append(1)
        else:
            l_wer.append(jiwer.wer(whisper_text, gt[i]))
        l_res_lang.append(res['language'])
        l_avg_logprob.append(res['avg_logprob'])
        l_no_speech_prob.append(res['no_speech_prob'])
        l_compres_ratio.append(res['compression_ratio'])
        
        
    df_tmp     = pd.DataFrame({
        "whisper":           l_whisper,
        "gt":                gt,
        "wer":               l_wer,
        "main_file":         batch['main_file'],
        "sub_file":          batch['sub_file'],
        "full_path":         batch['full_path'],
        "detect_lang":       l_res_lang,
        "avg_logprob":       l_avg_logprob,
        "no_speech_prob":    l_no_speech_prob,
        "compression_ratio": l_compres_ratio,
        
    })
    df = pd.concat([df, df_tmp], ignore_index=True)
    df.to_csv("/home/amitli/Datasets/whisper.csv")
    
                        

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1180/1180 [4:01:33<00:00, 12.28s/it]


In [137]:
df.head()

,whisper,gt,wer,main_file,sub_file,full_path,detect_lang,avg_logprob,no_speech_prob,compression_ratio
0,שלום לכם אני דוקטור שי שכיר והיום אנחנו הולכים...,שלום לכם שמי דוקטור שי שפיר והיום אנחנו הולכים...,0.200000,2RpZkeaTIm4,2RpZkeaTIm4_0000.wav,/home/amitli/Datasets/MyVoiceTests_16SR/2RpZke...,he,-0.333950,0.061862,1.256410
1,שכולנו מרגישים אותו ואיך זה משפיע לא רק על התח...,שכולם מרגישים אותו ואיך זה משפיע לא רק על הסביבה,0.272727,2RpZkeaTIm4,2RpZkeaTIm4_0001.wav,/home/amitli/Datasets/MyVoiceTests_16SR/2RpZke...,he,-0.519936,0.473480,1.232558
2,אלף זה משפיע בעיקר על הסביבה המהנית,היבשתית איך זה משפיע בעיקר על הסביבה המימית אז...,0.714286,2RpZkeaTIm4,2RpZkeaTIm4_0002.wav,/home/amitli/Datasets/MyVoiceTests_16SR/2RpZke...,he,-0.365871,0.476426,1.065574
3,כדור הערב הוא אוקיינוס הוא ים וזה מאוד משמעותי כי,כדור הארץ הוא אוקיינוס הוא ים וזה מאוד משמעותי כי,0.100000,2RpZkeaTIm4,2RpZkeaTIm4_0003.wav,/home/amitli/Datasets/MyVoiceTests_16SR/2RpZke...,he,-0.410051,0.297643,1.236842
4,ובעיקר שטח הפנים וגם הנפח של כדור הארץ זה מים,עיקר שטח הפנים וגם הנפח של כדור הארץ זה מים,0.100000,2RpZkeaTIm4,2RpZkeaTIm4_0004.wav,/home/amitli/Datasets/MyVoiceTests_16SR/2RpZke...,he,-0.343489,0.105788,1.154930


<u> <p style="color:blue; font-size:24px"> Hebrew Detection:</p> </u>

In [150]:
len(df), round(np.sum(df['detect_lang'].values == 'he')/len(df), 2)

(23596, 0.92)

<u> <p style="color:blue; font-size:24px"> Empty whisper result: </p> </u>

In [158]:
round(len(df[df['whisper'] == ''])/len(df), 2)

0.08

In [146]:
np.sum(df[df['whisper'] == '']['detect_lang'].values == 'he')

4

<u> <p style="color:blue; font-size:24px"> Bad whisper results: </p> </u>

In [164]:
round(np.sum(df['wer'].values > 1)/len(df), 2)

0.06

In [163]:
round(np.sum(df['wer'].values > 0.7)/len(df), 2)

0.28

In [162]:
round(np.sum(df['wer'].values > 0.5)/len(df), 2)

0.42

In [168]:
round(np.sum(df['wer'].values >= 0.3)/len(df), 2)

0.73

<u> <p style="color:blue; font-size:24px"> View Bad whisper results: </p> </u>

In [172]:
df[df['wer'].values > 0.5][['gt', 'whisper']].head(10)

,gt,whisper
2,היבשתית איך זה משפיע בעיקר על הסביבה המימית אז...,אלף זה משפיע בעיקר על הסביבה המהנית
16,השנים האחרונות גורמת להמסה של קרחונים בכל הארץ...,גורמת להנחה של קרחונים בכל הארץ
20,פחות מי תהום לשתות שלא לדבר על שינוי אקלים גדול,הרמות מיתרונותו שלא לדבר על שינוי אקליני גדול
24,מי הים אבל יש אספקטים נוספים של ההתחממות הזאת ...,אבל יש אספקטים נוספים של ההסכמנות הזאת הגלובלית
25,מאוד על אצות זעירות שנמצאות לו בתוך הרקמות ועו...,על עשות שעירות שנמצאות לו בתוך הוויכמות ועוברו...
26,המולבנים מטריד וכואב,כמו לבלים נטריד וכואב
27,מסוים ימות פשוט מאוד מרעב לא יהיו לו,יחוים ימות פשוט מאוד מרעב
35,מפלס מי הים משפיעה על חום טמפרטורת המים כיצד ה...,נפלס מהים משפיעה על חום טמפרטורת המים כשתחתת ה...
41,זה כל אותם יצורים שמושפעים בתנועתם מזרימת המים...,וכל אותם ייחורים שמושפעים בתנועתם לזרימת המים
42,יצורים זעירים סרטנים זהירים דגיגים זעירים ביצי...,שרטונים זהירים שרטנים זהירים גיגים זהירים ביצי...
